<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EPAT Session 2

**Executive Program in Algorithmic Trading**

**_OOP & Event-Based Backtesting_**

Prof. Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

<a href="http://shop.oreilly.com/product/0636920269861.do" target="_blank"><img src="https://hilpisch.com/aiif_cover.png" width="300px" align="left" border="1px"></a>

## Basic Imports

In [ ]:
import time
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment', None)

## OOP

In [ ]:
pd.__version__  # package

In [ ]:
pd.DataFrame  # class from the package

In [ ]:
df = pd.DataFrame()  # instance of the class

In [ ]:
type(df)  # type of object

## Reading Financial Data

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'  # EOD data
# url = 'http://hilpisch.com/aiif_eikon_id_data.csv'  # intraday data

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
    def __init__(self, symbol):  # called during instantiation
        self.symbol = symbol  # instance attribute
        self.prepare_data()  # calls the method
    def prepare_data(self):  # instance method
        self.raw = pd.read_csv(self.url, index_col=0,
                               parse_dates=True).dropna()
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['r'] = np.log(self.data / self.data.shift(1))
    def plot_data(self, cols=None):
        if cols is None:
            cols = [self.symbol]
        self.data[cols].plot(figsize=(10, 6))

## Event-Based View/Approach

In [ ]:
for bar in range(10):
    print(bar, raw.index[bar], raw['EUR='].iloc[bar])
    time.sleep(1)

## Event-Based Backtesting (Base Class)

In [ ]:
class BacktestingBase(FinancialData):
    def __init__(self, symbol, amount, verbose=True):
        super(BacktestingBase, self).__init__(symbol)
        self.initial_balance = amount
        self.current_balance = amount
        self.verbose=verbose
        self.units = 0
        self.trades = 0
    def get_date_price(self, bar):
        date = str(self.data.index[bar])[:10]
        price = self.data[self.symbol].iloc[bar]
        return date, price
    def print_current_balance(self, bar):
        date, price = self.get_date_price(bar)
        print(f'{date} | current balance = {self.current_balance:.2f}')
    def print_net_wealth(self, bar):
        date, price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units * price
        print(f'{date} | net wealth = {net_wealth:.2f}')
    def place_buy_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance -= units * price
        self.units += units
        if self.verbose:
            print(f'{date} | buy  {units} units for {price}')
            self.print_current_balance(bar)
            self.print_net_wealth(bar)
        self.trades += 1
    def place_sell_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance += units * price
        self.units -= units
        if self.verbose:
            print(f'{date} | sell {units} units for {price}')
            self.print_current_balance(bar)
            self.print_net_wealth(bar)
        self.trades += 1
    def close_out(self, bar):
        date, price = self.get_date_price(bar)
        self.current_balance += self.units * price
        print(55 * '=')
        print(f'*** CLOSING OUT FINAL POSITION ***')
        print(55 * '=')
        print(f'{date} | closing out {self.units} for {price}')
        self.units = 0
        self.trades += 1
        aperf = self.current_balance - self.initial_balance
        rperf = aperf / self.initial_balance * 100
        self.print_current_balance(bar)
        self.print_net_wealth(bar)
        print(f'{date} | # of trades = {self.trades}')
        print(f'{date} | aperf = {aperf:.2f} | rperf = {rperf:.3f}[%]')
        print(55 * '=')

In [ ]:
bb = BacktestingBase('EUR=', 10000, verbose=True)

In [ ]:
bb.get_date_price(100)

In [ ]:
bb.print_current_balance(100)

In [ ]:
bb.place_buy_order(100, units=1000)

In [ ]:
bb.place_buy_order(200, amount=2500)

In [ ]:
bb.place_sell_order(500, units=500)

In [ ]:
bb.units

In [ ]:
bb.close_out(1000)

## Event-Based Backtesting (SMA Strategy)

In [ ]:
class SMABacktesterEvent(BacktestingBase):
    def prepare_statistics(self):
        self.data['SMA1'] = self.data[self.symbol].rolling(self.SMA1).mean()
        self.data['SMA2'] = self.data[self.symbol].rolling(self.SMA2).mean()
    def run_strategy(self, SMA1, SMA2):
        self.trades = 0
        self.position = 0
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.prepare_statistics()
        print(55 * '=')
        print(f'RUN BACKTEST | {self.symbol} | SMA1={self.SMA1} | SMA2={self.SMA2}')
        print(55 * '=')
        for bar in range(self.SMA2, len(self.data)):
            if self.position in [0, -1]:
                if self.data['SMA1'].iloc[bar] > self.data['SMA2'].iloc[bar]:
                    # place a buy order --> go long
                    self.place_buy_order(bar, units=(1 - self.position) * 5000)
                    self.position = 1
            elif self.position in [0, 1]:
                if self.data['SMA1'].iloc[bar] < self.data['SMA2'].iloc[bar]:
                    # place a sell order --> go short
                    self.place_sell_order(bar, units=(1 + self.position) * 5000)
                    self.position = -1
        self.close_out(bar)

In [ ]:
sma = SMABacktesterEvent('EUR=', 10000)

In [ ]:
sma.run_strategy(42, 252)


In [9]:
class MyClass:
    """A simple example class"""
    i = 12345
    @classmethod
    def f(cls):
        return 'hello world'
    @property
    def name(self):
        print('bullshit')
shit = MyClass()
shit.name

bullshit


<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>